In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/pakistansuicideattacks/PakistanSuicideAttacks Ver 6 (10-October-2017).csv
/kaggle/input/pakistansuicideattacks/PakistanSuicideAttacks Ver 11 (30-November-2017).csv


In [2]:
import pandas as pd
import numpy as np

import fuzzywuzzy
from fuzzywuzzy import process
import chardet

# set seed for reproducibility
np.random.seed(0)

In [3]:
pak=pd.read_csv("/kaggle/input/pakistansuicideattacks/PakistanSuicideAttacks Ver 11 (30-November-2017).csv")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 0: invalid start byte

In [4]:
f=open("/kaggle/input/pakistansuicideattacks/PakistanSuicideAttacks Ver 11 (30-November-2017).csv",'rb')
res=chardet.detect(f.read(100000))
res

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}

In [5]:
pak=pd.read_csv("/kaggle/input/pakistansuicideattacks/PakistanSuicideAttacks Ver 11 (30-November-2017).csv",encoding='Windows-1252')
pak.head()

,S#,Date,Islamic Date,Blast Day Type,Holiday Type,Time,City,Latitude,Longitude,Province,...,Targeted Sect if any,Killed Min,Killed Max,Injured Min,Injured Max,No. of Suicide Blasts,Explosive Weight (max),Hospital Names,Temperature(C),Temperature(F)
0,1,Sunday-November 19-1995,25 Jumaada al-THaany 1416 A.H,Holiday,Weekend,NaN,Islamabad,33.7180,73.0718,Capital,...,None,14.0,15.0,NaN,60,2.0,NaN,NaN,15.835,60.503
1,2,Monday-November 6-2000,10 SHa`baan 1421 A.H,Working Day,NaN,NaN,Karachi,24.9918,66.9911,Sindh,...,None,NaN,3.0,NaN,3,1.0,NaN,NaN,23.770,74.786
2,3,Wednesday-May 8-2002,25 safar 1423 A.H,Working Day,NaN,7:45 AM,Karachi,24.9918,66.9911,Sindh,...,Christian,13.0,15.0,20.0,40,1.0,2.5 Kg,1.Jinnah Postgraduate Medical Center 2. Civil ...,31.460,88.628
3,4,Friday-June 14-2002,3 Raby` al-THaany 1423 A.H,Working Day,NaN,11:10:00 AM,Karachi,24.9918,66.9911,Sindh,...,Christian,NaN,12.0,NaN,51,1.0,NaN,NaN,31.430,88.574
4,5,Friday-July 4-2003,4 Jumaada al-awal 1424 A.H,Working Day,NaN,NaN,Quetta,30.2095,67.0182,Baluchistan,...,Shiite,44.0,47.0,NaN,65,1.0,NaN,1.CMH Quetta \n2.Civil Hospital 3. Boland Medi...,33.120,91.616


In [6]:
city=pak["City"].unique()
city.sort()
city

array(['ATTOCK', 'Attock ', 'Bajaur Agency', 'Bannu', 'Bhakkar ', 'Buner',
       'Chakwal ', 'Chaman', 'Charsadda', 'Charsadda ', 'D. I Khan',
       'D.G Khan', 'D.G Khan ', 'D.I Khan', 'D.I Khan ', 'Dara Adam Khel',
       'Dara Adam khel', 'Fateh Jang', 'Ghallanai, Mohmand Agency ',
       'Gujrat', 'Hangu', 'Haripur', 'Hayatabad', 'Islamabad',
       'Islamabad ', 'Jacobabad', 'KURRAM AGENCY', 'Karachi', 'Karachi ',
       'Karak', 'Khanewal', 'Khuzdar', 'Khyber Agency', 'Khyber Agency ',
       'Kohat', 'Kohat ', 'Kuram Agency ', 'Lahore', 'Lahore ',
       'Lakki Marwat', 'Lakki marwat', 'Lasbela', 'Lower Dir', 'MULTAN',
       'Malakand ', 'Mansehra', 'Mardan', 'Mohmand Agency',
       'Mohmand Agency ', 'Mohmand agency', 'Mosal Kor, Mohmand Agency',
       'Multan', 'Muzaffarabad', 'North Waziristan', 'North waziristan',
       'Nowshehra', 'Orakzai Agency', 'Peshawar', 'Peshawar ', 'Pishin',
       'Poonch', 'Quetta', 'Quetta ', 'Rawalpindi', 'Sargodha',
       'Sehwan town',

 ### Inconsistencies in capitalizations and trailing white spaces are very common in text data and you can fix a good 80% of your text data entry inconsistencies by doing this.

In [7]:
pak['City']=pak['City'].str.title()
pak['City']=pak['City'].str.strip()

In [8]:
pak['City'].head()

0    Islamabad
1      Karachi
2      Karachi
3      Karachi
4       Quetta
Name: City, dtype: object

In [9]:
a=(pak['Province'].unique())
print(a)
type(a)
np.sort(a)

['Capital' 'Sindh' 'Baluchistan' 'Punjab' 'FATA' 'KPK' 'AJK' 'Fata'
 'Balochistan']


array(['AJK', 'Balochistan', 'Baluchistan', 'Capital', 'FATA', 'Fata',
       'KPK', 'Punjab', 'Sindh'], dtype=object)

In [10]:
pak['Province']=pak['Province'].str.title()
pak['Province']=pak['Province'].str.strip()

In [11]:
pak['Province'].head()

0        Capital
1          Sindh
2          Sindh
3          Sindh
4    Baluchistan
Name: Province, dtype: object

### Fuzzy matching: The process of automatically finding text strings that are very similar to the target string. In general, a string is considered "closer" to another one the fewer characters you'd need to change if you were transforming one string into another. So "apple" and "snapple" are two changes away from each other (add "s" and "n") while "in" and "on" and one change away (rplace "i" with "o"). You won't always be able to rely on fuzzy matching 100%, but it will usually end up saving you at least a little time.

Fuzzywuzzy returns a ratio given two strings. The closer the ratio is to 100, the smaller the edit distance between the two strings. Here, we're going to get the ten strings from our list of cities that have the closest distance to "d.i khan".

In [12]:
# get the top 10 closest matches to "d.i khan"
matches = fuzzywuzzy.process.extract("d.i khan", city, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

[('D. I Khan', 100),
 ('D.I Khan', 100),
 ('D.I Khan ', 100),
 ('D.G Khan', 88),
 ('D.G Khan ', 88),
 ('Khanewal', 50),
 ('Sudhanoti', 47),
 ('Hangu', 46),
 ('Kohat', 46),
 ('Kohat ', 46)]

In [13]:
# function to replace rows in the provided column of the provided dataframe
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [i[0] for i in matches if i[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

In [14]:
# use the function we just wrote to replace close matches to "d.i khan" with "d.i khan"
replace_matches_in_column(df=pak, column='City', string_to_match="d.i khan")

All done!


In [15]:
cities = pak['City'].unique()
a=np.sort(cities)
a

array(['Attock', 'Bajaur Agency', 'Bannu', 'Bhakkar', 'Buner', 'Chakwal',
       'Chaman', 'Charsadda', 'D.G Khan', 'Dara Adam Khel', 'Fateh Jang',
       'Ghallanai, Mohmand Agency', 'Gujrat', 'Hangu', 'Haripur',
       'Hayatabad', 'Islamabad', 'Jacobabad', 'Karachi', 'Karak',
       'Khanewal', 'Khuzdar', 'Khyber Agency', 'Kohat', 'Kuram Agency',
       'Kurram Agency', 'Lahore', 'Lakki Marwat', 'Lasbela', 'Lower Dir',
       'Malakand', 'Mansehra', 'Mardan', 'Mohmand Agency',
       'Mosal Kor, Mohmand Agency', 'Multan', 'Muzaffarabad',
       'North Waziristan', 'Nowshehra', 'Orakzai Agency', 'Peshawar',
       'Pishin', 'Poonch', 'Quetta', 'Rawalpindi', 'Sargodha',
       'Sehwan Town', 'Shabqadar-Charsadda', 'Shangla', 'Shikarpur',
       'Sialkot', 'South Waziristan', 'Sudhanoti', 'Sukkur', 'Swabi',
       'Swat', 'Taftan', 'Tangi, Charsadda District', 'Tank', 'Taunsa',
       'Tirah Valley', 'Totalai', 'Upper Dir', 'Wagah', 'Zhob',
       'd.i khan'], dtype=object)

In [16]:
# get the top 10 closest matches to "Kuram Agency"
matches = fuzzywuzzy.process.extract("Kuram Agency", city, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

[('Kuram Agency ', 100),
 ('KURRAM AGENCY', 96),
 ('Bajaur Agency', 72),
 ('Khyber Agency', 72),
 ('Khyber Agency ', 72),
 ('Orakzai Agency', 69),
 ('Mohmand Agency', 62),
 ('Mohmand Agency ', 62),
 ('Mohmand agency', 62),
 ('Mosal Kor, Mohmand Agency', 61)]

In [17]:
# use the function we just wrote to replace close matches to "Kuram Agency" 
replace_matches_in_column(df=pak, column='City', string_to_match="Kuram Agency")

All done!


In [18]:
cities = pak['City'].unique()
a=np.sort(cities)
a

array(['Attock', 'Bajaur Agency', 'Bannu', 'Bhakkar', 'Buner', 'Chakwal',
       'Chaman', 'Charsadda', 'D.G Khan', 'Dara Adam Khel', 'Fateh Jang',
       'Ghallanai, Mohmand Agency', 'Gujrat', 'Hangu', 'Haripur',
       'Hayatabad', 'Islamabad', 'Jacobabad', 'Karachi', 'Karak',
       'Khanewal', 'Khuzdar', 'Khyber Agency', 'Kohat', 'Kuram Agency',
       'Lahore', 'Lakki Marwat', 'Lasbela', 'Lower Dir', 'Malakand',
       'Mansehra', 'Mardan', 'Mohmand Agency',
       'Mosal Kor, Mohmand Agency', 'Multan', 'Muzaffarabad',
       'North Waziristan', 'Nowshehra', 'Orakzai Agency', 'Peshawar',
       'Pishin', 'Poonch', 'Quetta', 'Rawalpindi', 'Sargodha',
       'Sehwan Town', 'Shabqadar-Charsadda', 'Shangla', 'Shikarpur',
       'Sialkot', 'South Waziristan', 'Sudhanoti', 'Sukkur', 'Swabi',
       'Swat', 'Taftan', 'Tangi, Charsadda District', 'Tank', 'Taunsa',
       'Tirah Valley', 'Totalai', 'Upper Dir', 'Wagah', 'Zhob',
       'd.i khan'], dtype=object)